In [1]:
# 以dickionary的方式實作稀疏矩陣達到減少記憶體浪費，由於是無向圖
# 這裡用 function my_graph 來做查詢graph邊的控制。

INF = 1e+20 # 若無連線的通道權重設 INF

def my_graph(i, j):
    if (i, j) in graph:
        return graph[(i, j)]
    elif (j, i) in graph:
        return graph[(j, i)]
    else:
        return INF

    
graph = {}
spot = None    
with open('graph.txt', 'r') as f:
    n, m = f.readline().split(' ')
    n, m = int(n), int(m)
    spot = f.readline().split(' ')
    spot = [int(s) for s in spot]
    
    for i in range(m):
        i, j , w = [int(v) for v in f.readline().split(' ')]
        graph[(i-1, j-1)] = w  # 索引值-1讓點從 index: 0開始
print(graph)

        


{(0, 1): 10, (0, 2): 20, (0, 4): 5, (1, 2): 10, (1, 3): 20, (1, 4): 4, (2, 3): 4, (3, 4): 20}


In [2]:
# 按照題目的要求讀取query資料
s = None
query = None
with open('query.txt', 'r') as f:
    s = int(f.readline())-1
    query = [int(t) for t in f.readlines()]
    
print(query)
        
print(s)


[10, 30, 40, 60, 69, 70, 71, 999]
0


In [3]:
walked = [False]*n # 判斷該點是否經歷過
path = [] # 儲存已走過的路線
cycle = [] # 儲存確認會形成環的path

# 以path去查詢該path總共花多少時間
def path_time(path):
    s = ''
    t = 0
    for i in range(len(path)-1):
        t += spot[path[i]-1] #path的儲存index-1才是計算的index
        s += '+'+str(spot[path[i]-1])
        t += my_graph(path[i]-1, path[i+1]-1)
        s += '+'+str(my_graph(path[i]-1, path[i+1]-1))
    return t, s[1:] + '=' + str(t)
        


def sort_rule(x):
    return x['time'], len(x['path'])


# 以遞迴方式實作dfs，用dfs找出所有可以出發後又回到起點的環
# 其中出了起點以外的點不可重複遍歷。
def find_cycle(start, level):
    
    if level > 0:
        walked[start] = True
    path.append(start+1) # 加入路徑時 索引值+1 回復原圖索引值
    if start == s and level > 0:
        t, _ = path_time(path)
        # print('t = %d: cycle %s' %(t, str(path)), end='\n')
        cycle.append({'path': path.copy(), 'time': t})
    else:
        for i in range(n):
            if not walked[i] and my_graph(start, i) != INF:
                find_cycle(i, level+1)
                walked[i] = False
                del path[-1]

                
                
                
find_cycle(s, 0)
cycle.sort(key=sort_rule)
        
# 把環按照time由小到大排序後由後往前找出第一個time小於所需query的環          
for time in query:
    for i in range(len(cycle)-1, -2, -1):
        if time >= cycle[i]['time']:
            p = cycle[i]['path']
            print('t = %d: cycle %s in time: %s' %(time, str(p), path_time(p)[-1]))
            break
        elif i < 0:
            print('t = %d: infeasible' %(time))   

t = 10: infeasible
t = 30: infeasible
t = 40: cycle [1, 2, 1] in time: 20+10+0+10=40
t = 60: cycle [1, 5, 1] in time: 20+5+30+5=60
t = 69: cycle [1, 5, 2, 1] in time: 20+5+30+4+0+10=69
t = 70: cycle [1, 3, 2, 1] in time: 20+20+10+10+0+10=70
t = 71: cycle [1, 3, 2, 1] in time: 20+20+10+10+0+10=70
t = 999: cycle [1, 5, 4, 2, 3, 1] in time: 20+5+30+20+8+20+0+10+10+20=143
